# Read the CSV and Perform Basic Data Cleaning

In [1]:
import pandas as pd
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [2]:
# Set features. This will also be used as your x values. Try all features first
X = df.drop('koi_disposition', axis=1)
y = df['koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [4]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3563,0,0,0,0,10.548413,5.470000e-05,-5.470000e-05,139.064020,0.004110,-0.004110,...,-133,4.387,0.066,-0.123,1.092,0.181,-0.097,298.09543,44.737061,13.204
4099,0,0,0,0,24.754385,1.365000e-04,-1.365000e-04,140.207320,0.004460,-0.004460,...,-144,4.519,0.078,-0.052,0.804,0.056,-0.076,295.73535,42.576248,15.514
5460,0,0,0,0,1.057336,1.230000e-07,-1.230000e-07,131.792007,0.000096,-0.000096,...,-140,4.594,0.054,-0.027,0.683,0.054,-0.060,292.18417,49.310040,15.414
1091,0,0,0,0,201.118319,1.461000e-03,-1.461000e-03,187.569860,0.005290,-0.005290,...,-112,4.447,0.072,-0.108,0.954,0.135,-0.083,283.11377,48.131390,13.328
5999,0,0,0,0,91.649983,3.181000e-03,-3.181000e-03,175.715600,0.028600,-0.028600,...,-233,4.145,0.164,-0.164,1.608,0.905,-0.383,294.93198,39.812420,12.964


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [5]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# One-hot encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [6]:
from tensorflow.keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Dense
number_inputs = len(X.columns)
number_hidden_nodes = number_inputs + 1
number_outputs = len(y_train_categorical[0])
print(number_inputs, number_hidden_nodes, number_outputs)

model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
model.add(Dense(units=number_outputs, activation='softmax'))

#GridSearch (Gradient Descent) is being done in .compile or .fit)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y_train_categorical, epochs=1000,shuffle=True)

model_loss, model_accuracy = model.evaluate(X_train_scaled, y_train_categorical)
print(f"Train - Loss: {model_loss}, Accuracy: {model_accuracy}")

model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical)
print(f"Test - Loss: {model_loss}, Accuracy: {model_accuracy}")

#print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

00
164/164 [==============================] - 0s 877us/step - loss: 0.2113 - accuracy: 0.9089
Epoch 816/1000
164/164 [==============================] - 0s 880us/step - loss: 0.2077 - accuracy: 0.9068
Epoch 817/1000
164/164 [==============================] - 0s 887us/step - loss: 0.2062 - accuracy: 0.9151
Epoch 818/1000
164/164 [==============================] - 0s 870us/step - loss: 0.2096 - accuracy: 0.9105
Epoch 819/1000
164/164 [==============================] - 0s 862us/step - loss: 0.2128 - accuracy: 0.9073
Epoch 820/1000
164/164 [==============================] - 0s 871us/step - loss: 0.2034 - accuracy: 0.9154
Epoch 821/1000
164/164 [==============================] - 0s 871us/step - loss: 0.2129 - accuracy: 0.9059
Epoch 822/1000
164/164 [==============================] - 0s 880us/step - loss: 0.2022 - accuracy: 0.9109
Epoch 823/1000
164/164 [==============================] - 0s 889us/step - loss: 0.2150 - accuracy: 0.9052
Epoch 824/1000
164/164 [==============================] - 

In [8]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('neural.h5')